# Nano-U Research Analysis

This notebook provides tools for analyzing trained Nano-U models, visualizing their performance, and inspecting NAS (Neural Architecture Search) metrics.

In [ ]:
import os
import sys
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd
from pathlib import Path

# Add project root to path
project_root = Path("..")
sys.path.insert(0, str(project_root.resolve()))

from src.models import create_nano_u
from src.nas import analyze_model_redundancy
from src.data import get_synthetic_data
from src.benchmarks import benchmark_inference

## 1. Model Inspection

Load a trained model and inspect its architecture and parameter count.

In [ ]:
model = create_nano_u()
model.summary()
print(f"\nTotal Parameters: {model.count_params():,}")

## 2. NAS Redundancy Analysis

Analyze layer redundancy using SVD-based condition numbers. This helps identify over-parameterized layers that can be pruned for microcontroller deployment.

In [ ]:
x_sample, _ = get_synthetic_data(num_samples=32)
analysis = analyze_model_redundancy(model, x_sample)

# Visualize redundancy scores
layers = list(analysis['layer_metrics'].keys())
scores = [m['redundancy_score'] for m in analysis['layer_metrics'].values()]

plt.figure(figsize=(10, 5))
plt.bar(layers, scores, color='skyblue')
plt.xlabel('Layer Name')
plt.ylabel('Redundancy Score (Lower is better)')
plt.title('Layer Redundancy Analysis')
plt.ylim(0, 1.1)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

## 3. Performance Benchmarking

Measure inference latency and throughput on the current hardware.

In [ ]:
benchmarks = benchmark_inference(model)
print(f"Avg Latency: {benchmarks['avg_latency_ms']:.2f} ms")
print(f"Throughput: {benchmarks['throughput_fps']:.2f} FPS")

## 4. NAS Training History

If you have run experiments with NAS monitoring enabled, you can visualize the redundancy evolution here.

In [ ]:
# Example: loading from a results directory
# history_df = pd.read_csv('results/my_experiment/nas_logs/metrics.csv')
# history_df.plot(x='epoch', y=[c for c in history_df.columns if 'redundancy' in c])
# plt.title('Redundancy Evolution during Training')
# plt.show()